In [32]:
from tqdm.notebook import tqdm
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import Normalizer

import gensim
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim_models

import spacy 

pd.set_option('display.max_colwidth', None) 

In [2]:
# Данные были спаршены из 4-х новостных пабликов республики Татарстан
df = pd.read_csv(r"C:\Users\ilyal\OneDrive\Рабочий стол\hackaton\files\posts_new.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150300 entries, 0 to 150299
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   public_url  150300 non-null  object
 1   post_url    150300 non-null  object
 2   post_text   138004 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


In [5]:
df.isna().sum()

public_url        0
post_url          0
post_text     12296
dtype: int64

Всего получего 150300 обьектов, из них 12296 были собраны некорректно, либо содержали в себе только фотографию 

In [6]:
df.duplicated().sum()

0

Дубликатов в исходных данных нет

Дропнем пропуски

In [7]:
df.dropna(inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138004 entries, 0 to 150299
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   public_url  138004 non-null  object
 1   post_url    138004 non-null  object
 2   post_text   138004 non-null  object
dtypes: object(3)
memory usage: 4.2+ MB


Теперь всё ок

Очистим текст от различных артефактов

In [9]:
def clean_text(text):
    text = text.replace('\n', ' ')  # удаление новых строк
    # удаление ссылок
    text = re.sub(r'http\S+', '', text) 
    text = re.sub(r'www\.\S+', '', text)
    text = re.sub(r'\S+@\S+\.\S+', '', text) # удаление email адресов
    text = re.sub(r'#\w+', '', text)
    text = re.sub('[^а-яА-ЯёЁ ]', ' ', text)  # оставляем только латинские буквы
    text = text.lower()  # приведение к нижнему регистру
    return text

In [10]:
df['cleaned_texts'] = df['post_text'].apply(clean_text)

Лемматизируем текст с помощью Spacy

In [13]:
# Загрузка модели русского языка для SpaCy
nlp = spacy.load("ru_core_news_sm")

def lem_tok_text(text):
    # Проверка, является ли текст строкой
    if isinstance(text, str):
        
        # Обработка текста с помощью SpaCy
        doc = nlp(text)
        
        # Удаление стоп-слов и лемматизация
        lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        
        # Возвращаем строку из лемматизированных слов
        return ' '.join(lemmatized_tokens)
    else:
        return ''  # Возвращаем пустую строку для значений, которые не являются строками

In [14]:
tqdm.pandas()
df['processed_text'] = df['cleaned_texts'].progress_apply(lem_tok_text)

  0%|          | 0/138004 [00:00<?, ?it/s]

In [46]:
# Функция для фильтрации слов
def remove_short_words(text):
    # Разделяем текст на слова, фильтруем и присоединяем обратно
    return ' '.join([word for word in text.split() if len(word) >= 3])

# Применяем функцию к столбцу
df['processed_text'] = df['processed_text'].apply(remove_short_words)

In [95]:
df = df[df['processed_text'].apply(lambda x: len(x.split()) >= 10)]

In [47]:
documents = df['processed_text'].tolist()

In [34]:
dictionary = corpora.Dictionary([doc.split() for doc in documents])

In [35]:
corpus = [dictionary.doc2bow(doc.split()) for doc in documents] 

In [25]:
n_clusters = 25 
lda_model = gensim.models.LdaModel(corpus, num_topics=n_clusters, id2word=dictionary, random_state=42)

In [48]:
# Выберите подвыборку данных
subset_documents = documents[:50000] 
dictionary = corpora.Dictionary([doc.split() for doc in subset_documents])
corpus = [dictionary.doc2bow(doc.split()) for doc in subset_documents]

In [49]:
n_clusters = 25 
lda_model = gensim.models.LdaModel(corpus, num_topics=n_clusters, id2word=dictionary, random_state=42)

In [50]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis, r"C:\Users\ilyal\OneDrive\Рабочий стол\hackaton\files\lda_visualisation_50k.html") 

---